# Chocolate Rating Analysis

## Web scraping data to then process, clean, etc. in the notebook.

In [4]:
# import warnings
# warnings.filterwarnings('ignore')

# from requests import get
# from bs4 import BeautifulSoup
# import pandas as pd
# from selenium import webdriver
# import chromedriver_autoinstaller
# def create_driver(headless=True):
#     driver = chromedriver_autoinstaller.install(cwd=True)
#     chrome_options = webdriver.ChromeOptions()     
#     if headless:
#         chrome_options.add_argument("--headless")
#         driver = webdriver.Chrome(driver, 
#                               chrome_options = chrome_options)
#         return driver
# driver = create_driver()

In [2]:
# driver.get("http://flavorsofcacao.com/chocolate_database.html")

In [3]:
# div_id = "spryregion1"

# html = driver.page_source
# soup = BeautifulSoup(html)
# div = soup.find('div', {'id':div_id})
# div

# table = div.find('table')
# table


In [4]:
# pd.read_html(str(table))[0]

In [7]:
df.to_csv('../../../data/chocolate.csv', index=False)

## Importing Necessary Python Libraries 

In [211]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import plot_confusion_matrix, auc, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.over_sampling import SMOTE

df = pd.read_csv('../../../data/chocolate.csv')
df.head()

# new data frame with split value columns 


,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


## Data Cleaning 

### Replacing column spaces with underscores and lower casing column names for simplicity when calling them. 

In [204]:
df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.lower()

df

,ref,company_manufacturer,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,ingredients,most_memorable_characteristics,rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50
...,...,...,...,...,...,...,...,...,...,...
2357,1205,Zotter,Austria,2014,Blend,Raw,80%,"4- B,S*,C,Sa","waxy, cloying, vegetal",2.75
2358,1996,Zotter,Austria,2017,Colombia,"APROCAFA, Acandi",75%,"3- B,S,C","strong nutty, marshmallow",3.75
2359,2170,Zotter,Austria,2018,Belize,Maya Mtn,72%,"3- B,S,C","muted, roasty, accessible",3.50
2360,2170,Zotter,Austria,2018,Congo,Mountains of the Moon,70%,"3- B,S,C","fatty, mild nuts, mild fruit",3.25


### Clarifying important predictor values (x) for the target value (y).

In [177]:
features = df.drop('rating', axis =1)
X = features
y = df.rating



### Splitting the data into three subsets of training and validation data for the future models.
    Two train test splits create three subsets of the original dataset which allows for the training data to not be bled into the test data - this reduced model's bias towards the pre-existing testing data, thus assuring maximum performance on future test sets which the model has never been exposed. 

In [178]:
#performing train test split for test set (subsets 1/3)
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=.15, random_state=42)


#performing a train test split for train and validation set (subsets - 3/3)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=.15, random_state=42) 

In [179]:
#checking to ensure the shape of the columns and rows are still the same for the X and y values after the train test split.
X_tr.shape, y_tr.shape

((2007, 9), (2007,))

### Replacing any existing missing values  

In [180]:
#ingredients have 88 rows that have no imputs.
df.isna().sum()

ref                                  0
company_manufacturer                 0
company_location                     0
review_date                          0
country_of_bean_origin               0
specific_bean_origin_or_bar_name     0
cocoa_percent                        0
ingredients                         88
most_memorable_characteristics       0
rating                               0
dtype: int64

In [181]:
# the empty value replacement will be done using the 
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [182]:
imputer.fit(X_train)

SimpleImputer(strategy='most_frequent')

In [183]:
X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X.columns)
X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X.columns)

In [184]:
print("X_train missing: ", X_train_imputed.isna().sum().sum())
print('X_val missing: ', X_val_imputed.isna().sum().sum())

X_train missing:  0
X_val missing:  0


In [185]:
# WHY DO WE USE  X INSTEAD OF X_TRAIN  

In [186]:
num_cols_train = list(X.select_dtypes('number').columns)
num_cols_train

['ref', 'review_date']

In [187]:
cat_cols_train = list(X.select_dtypes('object').columns)


In [188]:
num_cols_val = list(X.select_dtypes('number').columns)
num_cols_val

['ref', 'review_date']

In [189]:
cat_cols_val = list(X.select_dtypes('object').columns)
cat_cols_val

['company_manufacturer',
 'company_location',
 'country_of_bean_origin',
 'specific_bean_origin_or_bar_name',
 'cocoa_percent',
 'ingredients',
 'most_memorable_characteristics']

In [190]:
# Instantiate StandardScaler
scaler = StandardScaler()

In [191]:
# fit on Imputed training data with numeric columns in train
scaler.fit(X_train_imputed[num_cols_train])

StandardScaler()

In [192]:
# fit on Imputed training data with numeric  columns in val
scaler.fit(X_train_imputed[num_cols_val])

StandardScaler()

In [193]:
# transform training and validation data
X_train_scaled = pd.DataFrame(scaler.transform(X_train_imputed[num_cols_train]), columns=num_cols)
X_val_scaled = pd.DataFrame(scaler.transform(X_val_imputed[num_cols_val]), columns=num_cols)

In [194]:
X_train_scaled

,ref,review_date
0,-0.123022,0.022585
1,0.753333,0.557400
2,-1.511621,-1.581862
3,-0.100871,0.022585
4,0.130332,0.022585
...,...,...
1700,0.548435,0.557400
1701,0.302003,0.289992
1702,-0.978609,-0.779639
1703,0.504132,0.557400


In [195]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [196]:
# fit on training categorical data
ohe.fit(X_train_imputed[cat_cols_train])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [197]:
# fit on validation categorical data
ohe.fit(X_val_imputed[cat_cols_val])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [198]:
# transform training and validation data
X_train_encoded = ohe.transform(X_train_imputed[cat_cols_train], )
X_val_encoded = ohe.transform(X_val_imputed[cat_cols_val])

In [199]:
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=ohe.get_feature_names())
X_val_encoded_df = pd.DataFrame(X_val_encoded, columns=ohe.get_feature_names())
X_train_encoded_df

,x0_A. Morin,x0_Adi aka Fijiana (Easy In Ltd),x0_Aelan,x0_Aequare (Gianduja),x0_Akesson's (Pralus),x0_Alain Ducasse,x0_Alexandre,x0_Altus aka Cao Artisan,x0_Amazing Cacao,x0_Ambrosia,...,"x6_vanilla, dried fruit, nuts","x6_vegetal, herbs, raw","x6_very sandy, spicey, sweet","x6_very sweet, basic cocoa","x6_very sweet, coffee, mild sour","x6_very sweet, marshmallow","x6_very sweet, very spicey","x6_walnuts, cocoa, easy melt","x6_woody, butterscotch","x6_woody, nutty, short"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [200]:
X_train_encoded_df.loc[:].sum()

x0_A. Morin                         19.0
x0_Adi aka Fijiana (Easy In Ltd)     3.0
x0_Aelan                             3.0
x0_Aequare (Gianduja)                0.0
x0_Akesson's (Pralus)                1.0
                                    ... 
x6_very sweet, marshmallow           0.0
x6_very sweet, very spicey           0.0
x6_walnuts, cocoa, easy melt         0.0
x6_woody, butterscotch               0.0
x6_woody, nutty, short               0.0
Length: 882, dtype: float64

In [201]:
X_train_df = pd.concat([X_train_scaled, X_train_encoded_df], axis=1)
X_val_df = pd.concat([X_val_scaled, X_val_encoded_df], axis=1)

In [202]:
X_train_df.shape

(1705, 884)

In [203]:
X_val_df.shape

(302, 884)

## Preprocessing: